# 🔗 BƯỚC 0: GHÉP DỮ LIỆU (DATA MERGING)

## Tại sao cần ghép dữ liệu?

| File | Key liên kết | Cấu trúc |
|------|-------------|----------|
| `vietnam_destinations_normalized.csv` | date | 180 rows × 967 columns (wide format) |
| `vietnam_weather_monthly_2011_2025.csv` | province + date | 6,078 rows (long format) |
| `keyword_mapping_normalized.csv` | destination → province | Mapping table |

**Vấn đề**: Traffic data theo destination, Weather data theo province → Cần mapping để ghép

In [1]:
# =============================================================================
# STEP 0.1: LOAD TẤT CẢ DỮ LIỆU
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("📂 Loading all data files...")

# Main traffic data (wide format: rows=months, cols=destinations)
traffic_df = pd.read_csv('../data/normalized/vietnam_destinations_normalized.csv')

# Weather data (long format: rows=province+month)
weather_df = pd.read_csv('../data/normalized/vietnam_weather_monthly_2011_2025.csv')

# Mapping destination → province
mapping_df = pd.read_csv('../data/normalized/keyword_mapping_normalized.csv')

# Festivals data
festivals_df = pd.read_csv('../data/normalized/vietnam_festivals.csv')

# Seasonal patterns (117 destinations với strong seasonality)
seasonal_df = pd.read_csv('../data/normalized/vietnam_seasonal_destinations_strong.csv')

# Destinations statistics
stats_df = pd.read_csv('../data/normalized/destinations_statistics.csv')

print(f"\n✅ Traffic data: {traffic_df.shape} (rows=months, cols=destinations)")
print(f"✅ Weather data: {weather_df.shape}")
print(f"✅ Mapping data: {mapping_df.shape}")
print(f"✅ Festivals data: {festivals_df.shape}")
print(f"✅ Seasonal patterns: {seasonal_df.shape}")
print(f"✅ Statistics: {stats_df.shape}")

📂 Loading all data files...

✅ Traffic data: (180, 969) (rows=months, cols=destinations)
✅ Weather data: (6076, 6)
✅ Mapping data: (967, 6)
✅ Festivals data: (142, 13)
✅ Seasonal patterns: (117, 11)
✅ Statistics: (968, 8)


In [2]:
# =============================================================================
# STEP 0.2: CHUYỂN ĐỔI DATE FORMAT
# =============================================================================

def parse_vietnamese_date(date_str):
    """Chuyển 'thg 1 2011' → datetime"""
    months_vi = {
        'thg 1': 1, 'thg 2': 2, 'thg 3': 3, 'thg 4': 4,
        'thg 5': 5, 'thg 6': 6, 'thg 7': 7, 'thg 8': 8,
        'thg 9': 9, 'thg 10': 10, 'thg 11': 11, 'thg 12': 12
    }
    parts = date_str.rsplit(' ', 1)  # Split from right: ['thg 1', '2011']
    month_str, year = parts[0], int(parts[1])
    month = months_vi.get(month_str, 1)
    return pd.Timestamp(year=year, month=month, day=1)

# Convert traffic date column
traffic_df['date_parsed'] = traffic_df['date'].apply(parse_vietnamese_date)
print(f"📅 Date range: {traffic_df['date_parsed'].min()} to {traffic_df['date_parsed'].max()}")
print(f"   Total months: {len(traffic_df)}")

📅 Date range: 2011-01-01 00:00:00 to 2025-12-01 00:00:00
   Total months: 180


In [3]:
# =============================================================================
# STEP 0.3: CHUYỂN TRAFFIC DATA TỪ WIDE → LONG FORMAT
# =============================================================================
# Wide format: rows=months, cols=destinations
# Long format: rows=month×destination (để dễ merge với weather)

# Get destination columns (all except 'date' and 'date_parsed')
destination_cols = [col for col in traffic_df.columns if col not in ['date', 'date_parsed']]
print(f"📊 Number of destinations: {len(destination_cols)}")

# Melt to long format
traffic_long = traffic_df.melt(
    id_vars=['date_parsed'],
    value_vars=destination_cols,
    var_name='destination',
    value_name='traffic'
)

print(f"\n📊 Traffic data after melt:")
print(f"   Shape: {traffic_long.shape}")
print(f"   Columns: {traffic_long.columns.tolist()}")
traffic_long.head()

📊 Number of destinations: 968

📊 Traffic data after melt:
   Shape: (174240, 3)
   Columns: ['date_parsed', 'destination', 'traffic']


,date_parsed,destination,traffic
0,2011-01-01,Chợ Trung tâm Ba Chẽ,0.0
1,2012-01-01,Chợ Trung tâm Ba Chẽ,0.0
2,2013-01-01,Chợ Trung tâm Ba Chẽ,0.0
3,2014-01-01,Chợ Trung tâm Ba Chẽ,0.0
4,2015-01-01,Chợ Trung tâm Ba Chẽ,0.0


In [4]:
# =============================================================================
# STEP 0.4: GHÉP MAPPING (destination → province)
# =============================================================================

# Tạo mapping dict từ normalized_name → province_normalized
dest_to_province = dict(zip(
    mapping_df['normalized_name'], 
    mapping_df['province_normalized']
))

# Map province vào traffic_long
traffic_long['province'] = traffic_long['destination'].map(dest_to_province)

# Kiểm tra missing mappings
missing_mapping = traffic_long[traffic_long['province'].isna()]['destination'].unique()
print(f"⚠️ Destinations without province mapping: {len(missing_mapping)}")
if len(missing_mapping) > 0:
    print(f"   Examples: {missing_mapping[:5]}")

# Loại bỏ rows không có province mapping
traffic_long_clean = traffic_long.dropna(subset=['province'])
print(f"\n✅ Traffic data after mapping: {traffic_long_clean.shape}")

⚠️ Destinations without province mapping: 1
   Examples: ['Rau má']

✅ Traffic data after mapping: (174060, 4)


In [5]:
# =============================================================================
# STEP 0.5: GHÉP WEATHER DATA
# =============================================================================

# Convert weather date to same format
weather_df['date_parsed'] = pd.to_datetime(weather_df['date'])

# Merge traffic với weather theo province + date
merged_df = traffic_long_clean.merge(
    weather_df[['province', 'temp_avg', 'rainfall', 'date_parsed']],
    left_on=['province', 'date_parsed'],
    right_on=['province', 'date_parsed'],
    how='left'
)

print(f"✅ Merged data shape: {merged_df.shape}")
print(f"   Missing weather data: {merged_df['temp_avg'].isna().sum()} rows")

# Check sample
merged_df.head(10)

✅ Merged data shape: (174060, 6)
   Missing weather data: 1573 rows


,date_parsed,destination,traffic,province,temp_avg,rainfall
0,2011-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,12.106452,4.4
1,2012-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,14.035484,42.1
2,2013-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,14.680645,14.3
3,2014-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,15.161290,3.7
4,2015-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,15.532258,36.5
5,2016-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,15.670968,116.7
6,2017-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,16.587097,41.6
7,2018-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,14.332258,34.4
8,2019-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,15.025806,33.4
9,2020-01-01,Chợ Trung tâm Ba Chẽ,0.0,Quảng Ninh,16.600000,154.2


In [7]:
# =============================================================================
# STEP 0.6: THÊM THÔNG TIN SEASONALITY
# =============================================================================

# Column names in seasonal_df are different - check and use correct names
print(f"Seasonal columns: {seasonal_df.columns.tolist()}")

# Merge với seasonal patterns (using correct column names)
seasonal_info = seasonal_df[['Destination', 'Amplitude (Median Peak/Trough)', 'Peak_Months', 'Primary_Peak_Month']]
seasonal_info = seasonal_info.rename(columns={
    'Destination': 'destination',
    'Amplitude (Median Peak/Trough)': 'amplitude'
})

merged_df = merged_df.merge(seasonal_info, on='destination', how='left')

# Mark if destination has strong seasonality
merged_df['has_strong_seasonality'] = merged_df['amplitude'].notna()

print(f"✅ Destinations with strong seasonality: {merged_df['has_strong_seasonality'].sum() / len(merged_df) * 100:.1f}%")

# Add time features
merged_df['year'] = merged_df['date_parsed'].dt.year
merged_df['month'] = merged_df['date_parsed'].dt.month
merged_df['quarter'] = merged_df['date_parsed'].dt.quarter

print(f"\n📊 Final merged dataset:")
print(f"   Shape: {merged_df.shape}")
print(f"   Columns: {merged_df.columns.tolist()}")

Seasonal columns: ['Destination', 'Peak_Months', 'Strong_Months (>=1.2x)', 'Primary_Peak_Month', 'Amplitude (Median Peak/Trough)', 'CV (std/median of months)', 'Coverage OK Months', 'Có mùa vụ?', 'Peak_Months_List', 'Num_Strong_Months', 'Seasonal_Months_Clustered']
✅ Destinations with strong seasonality: 12.1%

📊 Final merged dataset:
   Shape: (174060, 13)
   Columns: ['date_parsed', 'destination', 'traffic', 'province', 'temp_avg', 'rainfall', 'amplitude', 'Peak_Months', 'Primary_Peak_Month', 'has_strong_seasonality', 'year', 'month', 'quarter']


In [8]:
# =============================================================================
# STEP 0.7: LƯU DỮ LIỆU ĐÃ GHÉP
# =============================================================================

# Lưu merged dataset
output_path = '../data/normalized/merged_tourism_data.csv'
merged_df.to_csv(output_path, index=False)
print(f"💾 Saved merged data to: {output_path}")

# Tóm tắt dữ liệu
print(f"\n" + "="*60)
print("📋 TÓM TẮT DỮ LIỆU ĐÃ GHÉP")
print("="*60)
print(f"📊 Total rows: {len(merged_df):,}")
print(f"📍 Unique destinations: {merged_df['destination'].nunique()}")
print(f"🗺️ Unique provinces: {merged_df['province'].nunique()}")
print(f"📅 Date range: {merged_df['date_parsed'].min()} to {merged_df['date_parsed'].max()}")
print(f"🌡️ Weather features: temp_avg, rainfall")
print(f"📈 Seasonality info: Amplitude, Peak_Months, Primary_Peak_Month")
print("="*60)

# Show sample
merged_df.sample(5)

💾 Saved merged data to: ../data/normalized/merged_tourism_data.csv

📋 TÓM TẮT DỮ LIỆU ĐÃ GHÉP
📊 Total rows: 174,060
📍 Unique destinations: 967
🗺️ Unique provinces: 34
📅 Date range: 2011-01-01 00:00:00 to 2025-12-01 00:00:00
🌡️ Weather features: temp_avg, rainfall
📈 Seasonality info: Amplitude, Peak_Months, Primary_Peak_Month


,date_parsed,destination,traffic,province,temp_avg,rainfall,amplitude,Peak_Months,Primary_Peak_Month,has_strong_seasonality,year,month,quarter
134833,2024-01-01,Bán đảo Đầm Môn,0.000000,Khánh Hoà,23.332258,11.6,NaN,NaN,NaN,False,2024,1,1
100727,2013-05-01,Đình Nại Nam,0.000000,TP. Đà Nẵng,28.945161,81.0,NaN,NaN,NaN,False,2013,5,2
85881,2017-10-01,Hồ Tuyền Lâm,11.111111,Lâm Đồng,22.112903,231.1,NaN,NaN,NaN,False,2017,10,4
77415,2011-10-01,Hòn Mây Rút,25.000000,An Giang,26.719355,194.7,NaN,NaN,NaN,False,2011,10,4
10001,2022-04-01,Đền Cái Lân,0.000000,Quảng Ninh,21.386667,85.6,NaN,NaN,NaN,False,2022,4,2


## 🔄 GHÉP THÊM DỮ LIỆU BỔ SUNG

Các file còn lại cần ghép:
- **Economic**: GRDP, Population, Area
- **Infrastructure**: Accommodation, Restaurant, Entertainment, Healthcare, Shop counts
- **Geographic**: Regions, Distances
- **Social Media**: YouTube views/likes
- **Statistics**: Destination statistics

In [9]:
# =============================================================================
# STEP 0.8: LOAD TẤT CẢ DỮ LIỆU BỔ SUNG
# =============================================================================

# Economic data
grdp_df = pd.read_csv('../data/normalized/vietnam_grdp_by_province.csv')
area_pop_df = pd.read_csv('../data/normalized/vietnam_area_population.csv')

# Geographic data  
regions_df = pd.read_csv('../data/normalized/vietnam_regions_with_distances.csv')

# Social media
youtube_df = pd.read_csv('../data/normalized/vietnam_youtube_province_aggregates.csv')

# Infrastructure counts (need to aggregate by province)
accommodation_df = pd.read_csv('../data/normalized/vietnam_accommodation.csv', sep=';')
restaurant_df = pd.read_csv('../data/normalized/vietnam_restaurants.csv', sep=';')
entertainment_df = pd.read_csv('../data/normalized/vietnam_entertainment.csv', sep=';')
healthcare_df = pd.read_csv('../data/normalized/vietnam_healthcare.csv', sep=';')
shops_df = pd.read_csv('../data/normalized/vietnam_shops.csv', sep=';')

# Destination statistics
dest_stats_df = pd.read_csv('../data/normalized/destinations_statistics.csv')

print("📂 Loaded additional data files:")
print(f"   GRDP: {grdp_df.shape}")
print(f"   Area/Population: {area_pop_df.shape}")
print(f"   Regions: {regions_df.shape}")
print(f"   YouTube: {youtube_df.shape}")
print(f"   Accommodation: {accommodation_df.shape}")
print(f"   Restaurant: {restaurant_df.shape}")
print(f"   Entertainment: {entertainment_df.shape}")
print(f"   Healthcare: {healthcare_df.shape}")
print(f"   Shops: {shops_df.shape}")
print(f"   Dest Stats: {dest_stats_df.shape}")

📂 Loaded additional data files:
   GRDP: (441, 4)
   Area/Population: (483, 6)
   Regions: (34, 10)
   YouTube: (36, 5)
   Accommodation: (11795, 3)
   Restaurant: (2151, 3)
   Entertainment: (339, 3)
   Healthcare: (329, 3)
   Shops: (597, 3)
   Dest Stats: (968, 8)


In [10]:
# =============================================================================
# STEP 0.9: TẠO INFRASTRUCTURE COUNTS BY PROVINCE
# =============================================================================

# Count infrastructure by province
infra_counts = pd.DataFrame()

# Accommodation count
acc_count = accommodation_df.groupby('province_normalized').size().reset_index(name='accommodation_count')

# Restaurant count
rest_count = restaurant_df.groupby('province_normalized').size().reset_index(name='restaurant_count')

# Entertainment count
ent_count = entertainment_df.groupby('province_normalized').size().reset_index(name='entertainment_count')

# Healthcare count
health_count = healthcare_df.groupby('province_normalized').size().reset_index(name='healthcare_count')

# Shop count
shop_count = shops_df.groupby('province_normalized').size().reset_index(name='shop_count')

# Merge all counts
infra_counts = acc_count.merge(rest_count, on='province_normalized', how='outer')
infra_counts = infra_counts.merge(ent_count, on='province_normalized', how='outer')
infra_counts = infra_counts.merge(health_count, on='province_normalized', how='outer')
infra_counts = infra_counts.merge(shop_count, on='province_normalized', how='outer')
infra_counts = infra_counts.fillna(0)

# Rename column
infra_counts = infra_counts.rename(columns={'province_normalized': 'province'})

print(f"📊 Infrastructure counts by province: {infra_counts.shape}")
infra_counts.head()

📊 Infrastructure counts by province: (33, 6)


,province,accommodation_count,restaurant_count,entertainment_count,healthcare_count,shop_count
0,An Giang,235,53,12.0,8.0,13
1,Bắc Ninh,132,35,0.0,0.0,12
2,Cao Bằng,164,31,0.0,0.0,4
3,Cà Mau,113,63,15.0,14.0,14
4,Gia Lai,187,36,6.0,6.0,12


In [11]:
# =============================================================================
# STEP 0.10: GHÉP REGIONS & GEOGRAPHIC DATA
# =============================================================================

# Prepare regions data (static by province)
regions_clean = regions_df[['province_normalized', 'region', 'latitude', 'longitude', 
                            'distance_to_hanoi_km', 'distance_to_hcm_km']].copy()
regions_clean = regions_clean.rename(columns={'province_normalized': 'province'})

# Merge regions into main dataset
merged_df = merged_df.merge(regions_clean, on='province', how='left')

print(f"✅ After merging regions: {merged_df.shape}")
print(f"   Columns: {merged_df.columns.tolist()}")

✅ After merging regions: (174060, 18)
   Columns: ['date_parsed', 'destination', 'traffic', 'province', 'temp_avg', 'rainfall', 'amplitude', 'Peak_Months', 'Primary_Peak_Month', 'has_strong_seasonality', 'year', 'month', 'quarter', 'region', 'latitude', 'longitude', 'distance_to_hanoi_km', 'distance_to_hcm_km']


In [12]:
# =============================================================================
# STEP 0.11: GHÉP INFRASTRUCTURE COUNTS
# =============================================================================

# Merge infrastructure counts into main dataset
merged_df = merged_df.merge(infra_counts, on='province', how='left')

# Fill NaN with 0 for infrastructure counts
infra_cols = ['accommodation_count', 'restaurant_count', 'entertainment_count', 
              'healthcare_count', 'shop_count']
merged_df[infra_cols] = merged_df[infra_cols].fillna(0)

print(f"✅ After merging infrastructure: {merged_df.shape}")

✅ After merging infrastructure: (174060, 23)


In [13]:
# =============================================================================
# STEP 0.12: GHÉP YOUTUBE DATA
# =============================================================================

# Prepare YouTube data
youtube_clean = youtube_df[['province_normalized', 'views', 'likes', 'comments']].copy()
youtube_clean = youtube_clean.rename(columns={
    'province_normalized': 'province',
    'views': 'youtube_views',
    'likes': 'youtube_likes',
    'comments': 'youtube_comments'
})

# Merge YouTube data
merged_df = merged_df.merge(youtube_clean, on='province', how='left')

print(f"✅ After merging YouTube: {merged_df.shape}")

✅ After merging YouTube: (189720, 26)


In [15]:
# =============================================================================
# STEP 0.13: GHÉP GRDP DATA (theo province + year)
# =============================================================================

# Check actual column names
print(f"GRDP columns: {grdp_df.columns.tolist()}")

# Clean GRDP data - find the GRDP column
grdp_col = [col for col in grdp_df.columns if 'GRDP' in col][0]
grdp_clean = grdp_df[['Năm', 'province_normalized', grdp_col]].copy()
grdp_clean.columns = ['year', 'province', 'grdp']

# Clean GRDP values (remove dots and commas, convert to float)
grdp_clean['grdp'] = grdp_clean['grdp'].astype(str).str.replace('.', '', regex=False)
grdp_clean['grdp'] = grdp_clean['grdp'].str.replace(',', '.', regex=False)
grdp_clean['grdp'] = pd.to_numeric(grdp_clean['grdp'], errors='coerce')

# Merge GRDP data by province + year
merged_df = merged_df.merge(grdp_clean, on=['province', 'year'], how='left')

print(f"✅ After merging GRDP: {merged_df.shape}")
print(f"   GRDP coverage: {merged_df['grdp'].notna().sum() / len(merged_df) * 100:.1f}%")

GRDP columns: ['Năm', 'Tên tỉnh, thành phố', 'Tổng GRDP\n\xa0(tỷ đồng)', 'province_normalized']
✅ After merging GRDP: (278256, 27)
   GRDP coverage: 63.6%


In [16]:
# =============================================================================
# STEP 0.14: GHÉP AREA & POPULATION DATA (theo province + year)
# =============================================================================

# Clean area/population data
area_pop_clean = area_pop_df[['Năm', 'province_normalized', 'Diện tích (Km2)', 
                              'Dân số trung bình (nghìn)', 'Mật độ dân số (người/km2)']].copy()
area_pop_clean.columns = ['year', 'province', 'area_km2', 'population_thousand', 'density']

# Merge area/population data by province + year
merged_df = merged_df.merge(area_pop_clean, on=['province', 'year'], how='left')

print(f"✅ After merging area/population: {merged_df.shape}")
print(f"   Population coverage: {merged_df['population_thousand'].notna().sum() / len(merged_df) * 100:.1f}%")

✅ After merging area/population: (544956, 30)
   Population coverage: 81.4%


In [17]:
# =============================================================================
# STEP 0.15: GHÉP DESTINATION STATISTICS & LOẠI BỎ DUPLICATES
# =============================================================================

# Loại bỏ duplicates nếu có
print(f"Before removing duplicates: {merged_df.shape}")
merged_df = merged_df.drop_duplicates(subset=['date_parsed', 'destination'])
print(f"After removing duplicates: {merged_df.shape}")

# Clean destination statistics
dest_stats_clean = dest_stats_df.rename(columns={
    'Destination': 'destination',
    'Mean': 'dest_mean_traffic',
    'Median': 'dest_median_traffic', 
    'Max': 'dest_max_traffic',
    'Std Dev': 'dest_std_traffic',
    'Coverage %': 'dest_coverage_pct'
})

# Merge destination statistics
merged_df = merged_df.merge(
    dest_stats_clean[['destination', 'dest_mean_traffic', 'dest_median_traffic', 
                      'dest_max_traffic', 'dest_std_traffic', 'dest_coverage_pct']], 
    on='destination', 
    how='left'
)

print(f"✅ After merging destination stats: {merged_df.shape}")

Before removing duplicates: (544956, 30)
After removing duplicates: (174060, 30)
✅ After merging destination stats: (174060, 35)


In [18]:
# =============================================================================
# STEP 0.16: TÓM TẮT VÀ LƯU DỮ LIỆU HOÀN CHỈNH
# =============================================================================

# Final summary
print("=" * 70)
print("📋 TÓM TẮT DỮ LIỆU ĐÃ GHÉP HOÀN CHỈNH")
print("=" * 70)
print(f"\n📊 Dataset shape: {merged_df.shape}")
print(f"📍 Unique destinations: {merged_df['destination'].nunique()}")
print(f"🗺️ Unique provinces: {merged_df['province'].nunique()}")
print(f"📅 Date range: {merged_df['date_parsed'].min()} to {merged_df['date_parsed'].max()}")

print(f"\n📋 ALL COLUMNS ({len(merged_df.columns)}):")
for i, col in enumerate(merged_df.columns):
    print(f"   {i+1}. {col}")

# Save complete merged dataset
output_path_full = '../data/normalized/merged_tourism_data_complete.csv'
merged_df.to_csv(output_path_full, index=False)
print(f"\n💾 Saved complete merged data to: {output_path_full}")

📋 TÓM TẮT DỮ LIỆU ĐÃ GHÉP HOÀN CHỈNH

📊 Dataset shape: (174060, 35)
📍 Unique destinations: 967
🗺️ Unique provinces: 34
📅 Date range: 2011-01-01 00:00:00 to 2025-12-01 00:00:00

📋 ALL COLUMNS (35):
   1. date_parsed
   2. destination
   3. traffic
   4. province
   5. temp_avg
   6. rainfall
   7. amplitude
   8. Peak_Months
   9. Primary_Peak_Month
   10. has_strong_seasonality
   11. year
   12. month
   13. quarter
   14. region
   15. latitude
   16. longitude
   17. distance_to_hanoi_km
   18. distance_to_hcm_km
   19. accommodation_count
   20. restaurant_count
   21. entertainment_count
   22. healthcare_count
   23. shop_count
   24. youtube_views
   25. youtube_likes
   26. youtube_comments
   27. grdp
   28. area_km2
   29. population_thousand
   30. density
   31. dest_mean_traffic
   32. dest_median_traffic
   33. dest_max_traffic
   34. dest_std_traffic
   35. dest_coverage_pct

💾 Saved complete merged data to: ../data/normalized/merged_tourism_data_complete.csv


In [19]:
# =============================================================================
# STEP 0.17: KIỂM TRA MISSING VALUES
# =============================================================================

# Check missing values
missing_summary = merged_df.isnull().sum()
missing_pct = (missing_summary / len(merged_df) * 100).round(2)

missing_df = pd.DataFrame({
    'Column': missing_summary.index,
    'Missing Count': missing_summary.values,
    'Missing %': missing_pct.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing %', ascending=False)

print("⚠️ MISSING VALUES SUMMARY:")
print(missing_df.to_string(index=False))

# Show sample of final dataset
print("\n📋 SAMPLE DATA:")
merged_df.sample(3)

⚠️ MISSING VALUES SUMMARY:
             Column  Missing Count  Missing %
          amplitude         153000      87.90
 Primary_Peak_Month         153000      87.90
        Peak_Months         153000      87.90
population_thousand          92832      53.33
               grdp          92832      53.33
            density          92832      53.33
           area_km2          92832      53.33
           temp_avg           1573       0.90
           rainfall           1573       0.90

📋 SAMPLE DATA:


,date_parsed,destination,traffic,province,temp_avg,rainfall,amplitude,Peak_Months,Primary_Peak_Month,has_strong_seasonality,...,youtube_comments,grdp,area_km2,population_thousand,density,dest_mean_traffic,dest_median_traffic,dest_max_traffic,dest_std_traffic,dest_coverage_pct
2594,2025-02-01,Thác Khe Vằn,0.0,Quảng Ninh,16.210714,39.4,NaN,NaN,NaN,False,...,316,NaN,NaN,NaN,NaN,0.168183,0.0,26.666667,1.995143,1.666667
82995,2011-10-01,Làng trầu Vị Thủy,0.0,TP. Cần Thơ,26.500000,190.7,NaN,NaN,NaN,False,...,3859,NaN,NaN,NaN,NaN,0.000000,0.0,0.000000,0.000000,0.000000
121552,2018-12-01,Suối Tiên Thành phố Hồ Chí Minh,0.0,TP. Hồ Chí Minh,27.206452,65.5,NaN,NaN,NaN,False,...,4517,322934.04,1981.0,1138.3,574.6,0.056321,0.0,2.380952,0.315810,3.333333


---
# ✅ HOÀN TẤT GHÉP DỮ LIỆU

Bây giờ bạn có thể sử dụng `merged_df` hoặc load từ file `merged_tourism_data.csv` cho các phân tích tiếp theo.

**Cấu trúc dữ liệu đã ghép:**
| Column | Description |
|--------|-------------|
| date_parsed | Ngày (monthly) |
| destination | Tên điểm đến |
| traffic | Lượng tìm kiếm |
| province | Tỉnh/TP |
| temp_avg | Nhiệt độ trung bình |
| rainfall | Lượng mưa |
| Amplitude | Biên độ dao động (seasonality) |
| Peak_Months | Tháng cao điểm |
| year, month, quarter | Time features |

---

### 1️⃣ DỮ LIỆU CHÍNH (Core Data)

| File | Mô tả | Số dòng | Columns chính |
|------|-------|---------|---------------|
| `vietnam_destinations_normalized.csv` | **Traffic tìm kiếm theo tháng** cho 967 địa điểm (2011-2025) | 180 rows | date, [967 destination columns] |
| `destinations_statistics.csv` | Thống kê tổng hợp cho mỗi địa điểm | 968 rows | Destination, Mean, Median, Max, Min, Std Dev, Non-zero Count, Coverage % |
| `keyword_mapping_normalized.csv` | Mapping địa điểm → tỉnh/thành | 967 rows | original_name, normalized_name, province, province_normalized |
| `vietnam_seasonal_destinations_strong.csv` | **Địa điểm có tính mùa vụ mạnh** | 117 rows | Destination, Peak_Months, Primary_Peak_Month, Amplitude, CV |

### 2️⃣ DỮ LIỆU BỔ TRỢ (Supporting Data)

| File | Mô tả | Dùng cho |
|------|-------|----------|
| `vietnam_weather_monthly_2011_2025.csv` | Thời tiết theo tháng (nhiệt độ, lượng mưa) | Feature thời tiết |
| `vietnam_festivals.csv` | Lễ hội theo tháng (2018-2025) | Feature lễ hội |
| `vietnam_youtube_province_aggregates.csv` | Views/Likes YouTube theo tỉnh | Feature social media |
| `vietnam_grdp_by_province.csv` | GRDP theo tỉnh | Feature kinh tế |
| `vietnam_population.csv` | Dân số theo tỉnh | Feature dân số |
| `vietnam_regions_with_distances.csv` | Khoảng cách từ HN/HCM | Feature địa lý |
| `vietnam_accommodation.csv` | Số lượng khách sạn/nhà nghỉ | Feature hạ tầng |
| `vietnam_restaurants.csv` | Số lượng nhà hàng | Feature hạ tầng |
| `vietnam_entertainment.csv` | Điểm vui chơi giải trí | Feature hạ tầng |

---
## 🔬 PHẦN 1: KHÁM PHÁ DỮ LIỆU (EDA)

In [ ]:
# Import thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Cài đặt hiển thị
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)

# Đường dẫn dữ liệu
DATA_PATH = '../data/normalized/'

In [ ]:
# 1. Load dữ liệu chính - Traffic tìm kiếm
df_traffic = pd.read_csv(DATA_PATH + 'vietnam_destinations_normalized.csv')

# Chuyển đổi cột date
# Format: 'thg X YYYY' -> datetime
def parse_vietnamese_date(date_str):
    """Chuyển đổi format 'thg 1 2011' thành datetime"""
    parts = date_str.split()
    month = int(parts[1])
    year = int(parts[2])
    return pd.Timestamp(year=year, month=month, day=1)

df_traffic['date'] = df_traffic['date'].apply(parse_vietnamese_date)
df_traffic = df_traffic.set_index('date')

print(f"📅 Khoảng thời gian: {df_traffic.index.min()} đến {df_traffic.index.max()}")
print(f"📍 Số địa điểm: {len(df_traffic.columns)}")
print(f"📊 Số tháng dữ liệu: {len(df_traffic)}")

In [ ]:
# 2. Load thống kê địa điểm
df_stats = pd.read_csv(DATA_PATH + 'destinations_statistics.csv')
print("\n📊 Phân phối Mean Traffic:")
print(df_stats['Mean'].describe())

# Phân loại địa điểm theo mức traffic
df_stats['traffic_level'] = pd.cut(
    df_stats['Mean'], 
    bins=[0, 1, 10, 50, 100, float('inf')],
    labels=['Rất thấp', 'Thấp', 'Trung bình', 'Cao', 'Rất cao']
)
print("\n📈 Phân loại địa điểm theo traffic:")
print(df_stats['traffic_level'].value_counts())

In [ ]:
# 3. Load dữ liệu mùa vụ
df_seasonal = pd.read_csv(DATA_PATH + 'vietnam_seasonal_destinations_strong.csv')
print(f"\n🗓️ Số địa điểm có tính mùa vụ mạnh: {len(df_seasonal)}")

# Top 10 địa điểm có tính mùa vụ mạnh nhất
print("\n🔥 Top 10 địa điểm có biên độ mùa vụ cao nhất:")
df_seasonal.nlargest(10, 'Amplitude (Median Peak/Trough)')[['Destination', 'Peak_Months', 'Amplitude (Median Peak/Trough)', 'Primary_Peak_Month']]

In [ ]:
# 4. Mapping địa điểm với tỉnh/thành
df_mapping = pd.read_csv(DATA_PATH + 'keyword_mapping_normalized.csv')
print("\n📍 Phân bố địa điểm theo tỉnh/thành (Top 15):")
print(df_mapping['province_normalized'].value_counts().head(15))

---
## 🔬 PHẦN 2: PHÂN TÍCH XU HƯỚNG

In [ ]:
# 2.1 Phân tích xu hướng tổng thể
# Tính tổng traffic theo tháng
total_traffic = df_traffic.sum(axis=1)

fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot 1: Xu hướng tổng
axes[0].plot(total_traffic.index, total_traffic.values, linewidth=2, color='blue')
axes[0].set_title('📈 Xu hướng Traffic Tìm Kiếm Du Lịch Tổng Thể (2011-2025)', fontsize=14)
axes[0].set_xlabel('Năm')
axes[0].set_ylabel('Tổng Traffic')
axes[0].axhline(y=total_traffic.mean(), color='red', linestyle='--', label=f'Trung bình: {total_traffic.mean():.0f}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Phân tích theo tháng
monthly_avg = df_traffic.groupby(df_traffic.index.month).mean().mean(axis=1)
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, 12))
bars = axes[1].bar(range(1, 13), monthly_avg.values, color=colors)
axes[1].set_title('📊 Traffic Trung Bình Theo Tháng', fontsize=14)
axes[1].set_xlabel('Tháng')
axes[1].set_ylabel('Traffic Trung Bình')
axes[1].set_xticks(range(1, 13))
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# 2.2 Phân tích YoY Growth (Year-over-Year)
# Tính growth rate năm sau so với năm trước
yearly_traffic = df_traffic.groupby(df_traffic.index.year).sum().sum(axis=1)
yearly_growth = yearly_traffic.pct_change() * 100

print("📈 Tăng trưởng Traffic YoY (%):")
for year, growth in yearly_growth.dropna().items():
    emoji = "🟢" if growth > 0 else "🔴"
    print(f"{emoji} {year}: {growth:.1f}%")

---
## 🔬 PHẦN 3: FEATURE ENGINEERING

In [ ]:
# 3.1 Tạo features từ dữ liệu thời tiết
df_weather = pd.read_csv(DATA_PATH + 'vietnam_weather_monthly_2011_2025.csv')
df_weather['date'] = pd.to_datetime(df_weather['date'])

# Tạo features thời tiết theo tỉnh và tháng
weather_features = df_weather.groupby(['province', df_weather['date'].dt.to_period('M')]).agg({
    'temp_avg': 'mean',
    'rainfall': 'sum'
}).reset_index()

print("📊 Features Thời Tiết:")
print(weather_features.head())

In [ ]:
# 3.2 Tạo features từ lễ hội
df_festivals = pd.read_csv(DATA_PATH + 'vietnam_festivals.csv')

# Đếm số lễ hội theo tỉnh và tháng
festival_columns = [col for col in df_festivals.columns if col.startswith('month_gregorian')]

# Chuyển đổi thành format dài
festival_long = df_festivals.melt(
    id_vars=['province', 'festival_name', 'type', 'province_normalized'],
    value_vars=festival_columns,
    var_name='year_col',
    value_name='month'
)

# Đếm số lễ hội
festival_long['month'] = pd.to_datetime(festival_long['month'])
festival_count = festival_long.groupby(['province_normalized', festival_long['month'].dt.to_period('M')]).size().reset_index(name='festival_count')

print("📊 Số lễ hội theo tỉnh và tháng:")
print(festival_count.head(10))

In [ ]:
# 3.3 Tạo features lag và rolling
def create_time_features(series, destination_name):
    """Tạo features thời gian cho một địa điểm"""
    df = pd.DataFrame()
    df['traffic'] = series
    
    # Lag features (tháng trước)
    for i in [1, 2, 3, 6, 12]:
        df[f'lag_{i}m'] = series.shift(i)
    
    # Rolling statistics
    df['rolling_mean_3m'] = series.rolling(window=3).mean()
    df['rolling_mean_6m'] = series.rolling(window=6).mean()
    df['rolling_std_3m'] = series.rolling(window=3).std()
    
    # Growth rate
    df['mom_growth'] = series.pct_change()  # Month-over-Month
    df['yoy_growth'] = series.pct_change(12)  # Year-over-Year
    
    # Seasonal features
    df['month'] = series.index.month
    df['quarter'] = series.index.quarter
    df['year'] = series.index.year
    
    # Cyclical encoding cho month
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    return df

# Demo với một địa điểm
sample_dest = 'Hồ Hoàn Kiếm'
if sample_dest in df_traffic.columns:
    sample_features = create_time_features(df_traffic[sample_dest], sample_dest)
    print(f"📊 Features cho {sample_dest}:")
    print(sample_features.tail(10))

---
## 🤖 PHẦN 4: XÂY DỰNG MÔ HÌNH DỰ ĐOÁN

### 📋 CHIẾN LƯỢC MÔ HÌNH

#### A. Các Mô Hình Đề Xuất:

| Mô hình | Ưu điểm | Nhược điểm | Phù hợp khi |
|---------|---------|------------|-------------|
| **SARIMA** | Bắt được tính mùa vụ, interpretable | Chỉ dùng 1 chuỗi, linear | Dữ liệu có mùa vụ rõ ràng |
| **Prophet** | Dễ dùng, tự động detect seasonality | Cần tune nhiều | Time series với trend và mùa vụ |
| **XGBoost** | Xử lý nhiều features, robust | Cần feature engineering | Có nhiều features ngoại sinh |
| **LSTM** | Bắt được pattern phức tạp | Cần nhiều data, khó tune | Dữ liệu lớn, pattern phức tạp |
| **Ensemble** | Kết hợp ưu điểm các mô hình | Phức tạp | Production system |

#### B. Chiến Lược Dự Đoán Ngắn Hạn (1-3 tháng):

1. **Mô hình cấp địa điểm**: Dự đoán từng destination riêng biệt
2. **Mô hình cấp tỉnh**: Aggregate theo tỉnh rồi dự đoán
3. **Mô hình kết hợp**: Dự đoán tổng + phân bổ theo tỉ lệ lịch sử

In [ ]:
# 4.1 Cài đặt thêm thư viện (nếu cần)
# !pip install prophet statsmodels xgboost scikit-learn

In [ ]:
# 4.2 Mô hình SARIMA cho dự đoán ngắn hạn
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

def build_sarima_model(series, train_size=0.8):
    """
    Xây dựng mô hình SARIMA cho một chuỗi thời gian
    """
    # Chia train/test
    n = len(series)
    train = series[:int(n * train_size)]
    test = series[int(n * train_size):]
    
    # Fit model với SARIMA(1,1,1)(1,1,1,12) - seasonal period = 12 tháng
    try:
        model = SARIMAX(
            train, 
            order=(1, 1, 1),
            seasonal_order=(1, 1, 1, 12),
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        fitted = model.fit(disp=False)
        
        # Dự đoán
        predictions = fitted.forecast(len(test))
        
        # Đánh giá
        mae = mean_absolute_error(test, predictions)
        rmse = np.sqrt(mean_squared_error(test, predictions))
        
        return {
            'model': fitted,
            'predictions': predictions,
            'actual': test,
            'mae': mae,
            'rmse': rmse
        }
    except Exception as e:
        return {'error': str(e)}

# Demo với một địa điểm có traffic cao
high_traffic_destinations = df_stats.nlargest(10, 'Mean')['Destination'].tolist()
print("🏆 Top 10 địa điểm có traffic cao nhất:")
for i, dest in enumerate(high_traffic_destinations, 1):
    mean_val = df_stats[df_stats['Destination'] == dest]['Mean'].values[0]
    print(f"  {i}. {dest}: {mean_val:.1f}")

In [ ]:
# 4.3 XGBoost với features đầy đủ
from sklearn.model_selection import TimeSeriesSplit

def prepare_ml_dataset(df_traffic, df_mapping, destination):
    """
    Chuẩn bị dataset cho ML model
    """
    if destination not in df_traffic.columns:
        return None
    
    series = df_traffic[destination]
    df = create_time_features(series, destination)
    
    # Thêm thông tin tỉnh/thành
    province_info = df_mapping[df_mapping['normalized_name'] == destination]
    if len(province_info) > 0:
        province = province_info['province_normalized'].values[0]
        df['province'] = province
    
    # Drop NaN
    df = df.dropna()
    
    return df

print("✅ Functions đã sẵn sàng cho training model")

---
## 📊 PHẦN 5: PHÂN TÍCH CHUYÊN SÂU CHO DỰ ĐOÁN NGẮN HẠN

In [ ]:
# 5.1 Xác định địa điểm có tiềm năng tăng trưởng trong kỳ tới
def identify_trending_destinations(df_traffic, df_seasonal, n_months=3):
    """
    Xác định địa điểm có khả năng có traffic cao trong n tháng tới
    dựa trên:
    1. Peak months của địa điểm đó
    2. Trend gần đây
    3. YoY comparison
    """
    current_month = datetime.now().month
    upcoming_months = [(current_month + i) % 12 or 12 for i in range(1, n_months + 1)]
    
    print(f"🗓️ Phân tích cho {n_months} tháng tới: {upcoming_months}")
    
    potential_destinations = []
    
    for _, row in df_seasonal.iterrows():
        dest = row['Destination']
        peak_months = eval(row['Peak_Months_List']) if isinstance(row['Peak_Months_List'], str) else []
        
        # Check nếu peak months nằm trong upcoming months
        overlap = set(peak_months) & set(upcoming_months)
        if overlap:
            potential_destinations.append({
                'destination': dest,
                'peak_months': peak_months,
                'upcoming_peak_months': list(overlap),
                'amplitude': row['Amplitude (Median Peak/Trough)'],
                'primary_peak': row['Primary_Peak_Month']
            })
    
    result_df = pd.DataFrame(potential_destinations)
    return result_df.sort_values('amplitude', ascending=False)

# Chạy phân tích
trending = identify_trending_destinations(df_traffic, df_seasonal, n_months=3)
print(f"\n🔥 Số địa điểm có peak trong 3 tháng tới: {len(trending)}")
print("\n📈 Top 20 địa điểm tiềm năng:")
trending.head(20)

In [ ]:
# 5.2 Phân tích correlation với các yếu tố
def analyze_traffic_drivers(df_traffic, df_weather, df_mapping, destination):
    """
    Phân tích các yếu tố ảnh hưởng đến traffic của một địa điểm
    """
    if destination not in df_traffic.columns:
        return None
    
    # Lấy province
    province_info = df_mapping[df_mapping['normalized_name'] == destination]
    if len(province_info) == 0:
        return None
    
    province = province_info['province_normalized'].values[0]
    
    # Lấy traffic
    traffic = df_traffic[destination].reset_index()
    traffic.columns = ['date', 'traffic']
    
    # Merge với weather
    weather_subset = df_weather[df_weather['province'] == province].copy()
    if len(weather_subset) == 0:
        return None
    
    # Tính correlation
    traffic['month'] = traffic['date'].dt.month
    weather_subset['month'] = pd.to_datetime(weather_subset['date']).dt.month
    
    # Thống kê đơn giản
    result = {
        'destination': destination,
        'province': province,
        'avg_traffic': traffic['traffic'].mean(),
        'peak_month': traffic.groupby('month')['traffic'].mean().idxmax(),
        'low_month': traffic.groupby('month')['traffic'].mean().idxmin()
    }
    
    return result

print("✅ Function analyze_traffic_drivers đã sẵn sàng")

---
## 🎯 PHẦN 6: ĐỀ XUẤT & KẾT LUẬN

### 📋 HƯỚNG PHÂN TÍCH CHI TIẾT

#### BƯỚC 1: Data Understanding & Cleaning
1. ✅ Load và khám phá tất cả datasets
2. ✅ Xử lý missing values
3. ✅ Chuẩn hóa tên tỉnh/thành
4. ✅ Merge các nguồn dữ liệu

#### BƯỚC 2: EDA Chuyên Sâu
1. Phân tích xu hướng tổng thể traffic
2. Phân tích mùa vụ theo tháng/quý
3. Phân tích theo vùng miền
4. Xác định địa điểm có tính mùa vụ mạnh
5. Phân tích correlation với thời tiết, lễ hội

#### BƯỚC 3: Feature Engineering
1. **Time features**: Lag, rolling mean/std, growth rates
2. **Seasonal features**: Month sin/cos, quarter, is_holiday
3. **External features**: Weather, festivals, GRDP
4. **Infrastructure features**: Số khách sạn, nhà hàng

#### BƯỚC 4: Modeling
1. **Baseline**: SARIMA cho từng địa điểm
2. **Advanced**: XGBoost/LightGBM với all features
3. **Ensemble**: Kết hợp nhiều mô hình
4. **Evaluation**: MAE, RMSE, MAPE

#### BƯỚC 5: Prediction & Deployment
1. Dự đoán traffic 1-3 tháng tới
2. Ranking địa điểm tiềm năng
3. Đề xuất chiến lược marketing
4. Dashboard monitoring

### 📊 METRICS ĐÁNH GIÁ

| Metric | Công thức | Ý nghĩa |
|--------|-----------|----------|
| **MAE** | Mean Absolute Error | Sai số trung bình tuyệt đối |
| **RMSE** | Root Mean Squared Error | Penalize lỗi lớn nhiều hơn |
| **MAPE** | Mean Absolute Percentage Error | Sai số % - dễ hiểu |
| **Hit Rate** | % địa điểm predicted top-K nằm trong actual top-K | Đánh giá ranking |

### 🎯 ỨNG DỤNG THỰC TẾ

1. **Marketing Campaign**: Target địa điểm có traffic predicted cao
2. **Resource Planning**: Chuẩn bị hạ tầng, nhân lực
3. **Content Creation**: Tạo content cho địa điểm sắp peak
4. **Promotion Timing**: Chạy promotion trước khi peak

---
## 🚀 NEXT STEPS

1. **Hoàn thiện EDA**: Phân tích sâu hơn từng loại địa điểm
2. **Build models**: Implement SARIMA, Prophet, XGBoost
3. **Validate**: Cross-validation, backtesting
4. **Production**: API, Dashboard
5. **Monitor**: Theo dõi và cập nhật model

---

**📝 Note**: Notebook này là template hướng dẫn. Cần chạy và tinh chỉnh theo dữ liệu thực tế.